In [16]:
# df = spark.sql("SELECT * FROM InFreGen.task LIMIT 1000")
# display(df)
# # 删除所有给定taskID的pic
# spark.sql("DELETE FROM InFreGen.pic WHERE taskID = {}".format(taskID))
# taskID = 19

StatementMeta(, , , Waiting, , Waiting)

In [17]:
from enum import Enum

import requests
from pyspark.sql import functions as F
import json

ACCESS_KEY = 'YOUR KEY HERE'


class Orientation(Enum):
    UNKNOWN = 0
    LANDSCAPE = 1
    PORTRAIT = 2
    SQUARISH = 3

    def get_lower_value(self):
        if self == Orientation.UNKNOWN:
            return ""
        return self.name.lower()


class UnsplashClient:
    def __init__(self, access_key):
        self.access_key = access_key
        self.base_url = 'https://api.unsplash.com/'
        self.headers = {
            'Accept-Version': 'v1',
            'Authorization': f'Client-ID {self.access_key}'
        }

    def get_random_image(self, query: str = None, orientation: Orientation = Orientation.UNKNOWN):
        url = self.base_url + 'photos/random'
        params = {
            'query': query,
            'orientation': orientation.get_lower_value()
        }
        response = requests.get(url, headers=self.headers, params=params)
        return response.json() if response.status_code == 200 else None

    def search_images(
            self, query: str,
            page: int = 1,
            per_page: int = 30,
            orientation: Orientation = Orientation.UNKNOWN
    ):
        url = self.base_url + 'search/photos'
        params = {
            'query': query,
            'page': page,
            'per_page': per_page,
        }
        if orientation != Orientation.UNKNOWN:
            params['orientation'] = orientation.get_lower_value()
        response = requests.get(url, headers=self.headers, params=params)
        print(response.status_code)
        # 400 error 
        return response.json() if response.status_code == 200 else None

    # 你可以在这里添加更多的方法，比如获取特定图片的详细信息等等。

client = UnsplashClient(ACCESS_KEY)


StatementMeta(, , , Waiting, , Waiting)

In [18]:
import random
# # Query to find rows where 'url' is an empty string
# result_df = spark.sql("SELECT * FROM InFreGen.task WHERE State = 0")

# # Show the result
# display(result_df)
# # Select the 'userInput' column from the result_df DataFrame
# user_input_df = result_df.select("userInput")

# # Collect the userInput values into a list
# user_input_values = user_input_df.collect()

# # Print the collected userInput values
# for row in user_input_values:
#     print(row['userInput'])

task_row = spark.sql("SELECT * FROM InFreGen.task WHERE taskID = {}".format(taskID)).collect()[0]
print(task_row)
# Row(taskID=12, userInput='I want to do medical image cancer detection', keyword=None, num=8, resolution='1024x1024', sizeChoice='small', State=0)
# use the client to search the pic by the task_row['keyword']
def searchPerTask(task_row) -> list[dict]:
    keywords = task_row['keyword'].split(',')
    print("keyword:", keywords)
    pic_table = []
    count = 0
    while count < task_row['num']:
        keyword = random.choice(keywords)

        print("Chosen keyword: " + keyword)

        search_results = client.search_images(query=keyword, page=1, per_page=30, orientation=Orientation.SQUARISH)
        try:
            image_urls = random.choice(search_results['results'])['urls']
        except:
            continue
        # add a new row to the pic table
        new_row = {
            'taskID': task_row['taskID'],
            'picID': count,
            'Resolution': task_row['resolution'],
            'sizeChoice': task_row['sizeChoice'],
            'operations': "",
            'operationsReturn': "",
            'url': json.dumps(image_urls),
            'State': 0,
            'originalPicPath': "",
            'curPicPath': "",
            'keywords': keyword,
            'finalPicPath': "",
        }
        pic_table.append(new_row)
        count += 1
    return pic_table
    
# inject the pic table with the pic_df
from pyspark.sql.types import IntegerType

pic_df = spark.createDataFrame(searchPerTask(task_row))
pic_df = pic_df.withColumn("taskID", pic_df["taskID"].cast(IntegerType()))
pic_df = pic_df.withColumn("picID", pic_df["picID"].cast(IntegerType()))
pic_df = pic_df.withColumn("State", pic_df["State"].cast(IntegerType()))
pic_df.write.format("delta").mode("append").saveAsTable("InFreGen.pic")

StatementMeta(, , , Waiting, , Waiting)

In [19]:
# df = spark.sql("SELECT * FROM InFreGen.pic LIMIT 1000")
# display(df)

StatementMeta(, , , Waiting, , Waiting)

In [20]:
# # 使用示例

# client = UnsplashClient(ACCESS_KEY)

# # 创建 Delta 表 pic
# # DeltaTable.create(spark) \
# #   .tableName("pic") \
# #   .addColumn("taskID", "INT") \
# #   .addColumn("picID", "INT") \
# #   .addColumn("Resolution", "STRING") \
# #   .addColumn("sizeChoice", "String") \
# #   .addColumn("operations", "STRING") \
# #   .addColumn("operationsReturn", "STRING") \
# #   .addColumn("url", "STRING") \
# #   .addColumn("originalPicPath", "STRING") \
# #   .addColumn("State", "INT") \
# #   .addColumn("curPicPath", "STRING") \
# #   .addColumn("finalPicPath", "STRING") \
# #   .execute()
# import json
# from pyspark.sql import functions as F
# from pyspark.sql import Row

# # Query to find rows where 'url' is an empty string
# # Collect all necessary values
# task_values = result_df.collect()

# # Print the collected userInput values and create pic_table
# pic_table = []
# for row in task_values:
#     print(row['userInput'])
#     if row['userInput'] == "I want to do instance segmentation":
#         keyword = "Vase"
#     elif row['userInput'] == "I want to do semantic segmentation":
#         keyword = "Car"
#     elif row['userInput'] == "I want to do text to image":
#         keyword = "Flower"
#     elif row['userInput'] == "I want to do image captioning":
#         keyword = "Dog"
#     elif row['userInput'] == "I want to do Cat and Dog Classification":
#         keyword = "Cat"
#     elif row['userInput'] == "I want to do Animal Classification":
#         keyword = "Dog" 
#     else:
#         keyword = "Cat"     
#     search_results = client.search_images(query=keyword, page=1, per_page=30, orientation=Orientation.SQUARISH)

#     if search_results and 'results' in search_results and len(search_results['results']) > 0:
#         for i in range(row['num']):
#             # create new row as a dictionary
#             new_row = {
#                 'taskID': row['taskID'],
#                 'picID': i,
#                 'Resolution': "1024x1024",
#                 'sizeChoice': "small",
#                 'operations': "",
#                 'operationsReturn': "",
#                 'url': "",
#                 'originalPicPath': "",
#                 'State': 1,
#                 'curPicPath': "",
#                 'finalPicPath': ""
#             }
#             image_urls = search_results['results'][i]['urls']

#             # Serialize image URLs data as a dictionary
#             serialized_url_data = {
#                 'raw': image_urls.get('raw'),
#                 'full': image_urls.get('full'),
#                 'regular': image_urls.get('regular'),
#                 'small': image_urls.get('small'),
#                 'thumb': image_urls.get('thumb'),
#                 'small_s3': image_urls.get('small_s3')
#             }
#             # Convert the dictionary into a string representation
#             new_row['url'] = json.dumps(serialized_url_data)
#             pic_table.append(new_row)

# # Convert pic_table to a DataFrame
# pic_df = spark.createDataFrame(pic_table)
# display(pic_df)

StatementMeta(, , , Waiting, , Waiting)

In [21]:
# from pyspark.sql import functions as F
# from delta.tables import DeltaTable
# from pyspark.sql.types import StringType
# from pyspark.sql.types import IntegerType
# delta_table = DeltaTable.forName(spark, "InFreGen.pic")

# # # Perform the MERGE operation to update rows based on taskID and picID
# # delta_table.alias("target") \
# #     .merge(
# #         result_df.alias("source"),
# #         "target.taskID = source.taskID AND target.picID = source.picID"  # Condition for matching rows
# #     ) \
# #     .whenMatchedUpdate(
# #         condition="target.url != source.url",  # Only update if the URL is different
# #         set={"url": "source.url"}  # Update the URL column with the new value
# #     ) \
# #     .whenNotMatchedInsert(
# #         values={"taskID": "source.taskID", "picID": "source.picID", "url": "source.url"}  # Insert new row if no match
# #     ) \
# #     .execute()

# # # Show the updated table content (optional)
# # delta_table.toDF().show(truncate=False)
# # inject the pic table with the pic_df
# # inject the pic table with the pic_df
# pic_df = pic_df.withColumn("taskID", pic_df["taskID"].cast(IntegerType()))
# pic_df = pic_df.withColumn("picID", pic_df["picID"].cast(IntegerType()))
# pic_df = pic_df.withColumn("State", pic_df["State"].cast(IntegerType()))
# pic_df.write.format("delta").mode("append").saveAsTable("InFreGen.pic")


StatementMeta(, , , Waiting, , Waiting)

In [22]:
# df = spark.sql("SELECT * FROM InFreGen.pic LIMIT 1000")
# display(df)

StatementMeta(, , , Waiting, , Waiting)